In [21]:
import pandas as pd

data_filename = './NBA_Data.csv'
dataset = pd.read_csv(data_filename,parse_dates = ['Date'])
dataset.columns = ["Date", "Start (ET)", "Visitor Team", "VisitorPts", "Home Team", "HomePts",  "Score Type","OT?","Attend","Notes"]
dataset = dataset.sort_values('Date',ascending = True , axis=0)
dataset.head()

,Date,Start (ET),Visitor Team,VisitorPts,Home Team,HomePts,Score Type,OT?,Attend,Notes
0,2013-10-29,7:00 pm,Orlando Magic,87,Indiana Pacers,97,Box Score,NaN,18165,NaN
1,2013-10-29,10:30 pm,Los Angeles Clippers,103,Los Angeles Lakers,116,Box Score,NaN,18997,NaN
2,2013-10-29,8:00 pm,Chicago Bulls,95,Miami Heat,107,Box Score,NaN,19964,NaN
15,2013-10-30,7:00 pm,Boston Celtics,87,Toronto Raptors,93,Box Score,NaN,20155,NaN
14,2013-10-30,8:30 pm,Memphis Grizzlies,94,San Antonio Spurs,101,Box Score,NaN,18581,NaN


In [22]:
dataset['Homewin'] = dataset['HomePts']>dataset['VisitorPts']
Home_win = dataset['Homewin'].values


In [23]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.cross_validation import cross_val_score
import numpy as np
from collections import defaultdict
dataset["HomeLastWin"] = False
dataset["VisitorLastWin"] = False


In [24]:
won_in_last_game = defaultdict(bool)
for index , row in dataset.iterrows():
    visitor_team = row['Visitor Team']
    home_team = row['Home Team']
    #won_in_last_game初始化为false ，即在df中第一次出现的队伍主客场战绩都会被标记为False
    dataset.at[index , "VisitorLastWin"] = won_in_last_game[visitor_team]
    dataset.at[index , "HomeLastWin"] = won_in_last_game[home_team]
    #每次更新本次主场的胜负作为下一次遍历时的“上一场战绩”，主队胜则客队败
    won_in_last_game[home_team] = row['Homewin']
    won_in_last_game[visitor_team] = not row['Homewin']
Y_true = dataset['Homewin']
#dataset[(dataset['Home Team']=='San Antonio Spurs')|(dataset['Visitor Team']=='San Antonio Spurs')]
dataset.loc[20:25]

,Date,Start (ET),Visitor Team,VisitorPts,Home Team,HomePts,Score Type,OT?,Attend,Notes,Homewin,HomeLastWin,VisitorLastWin
20,2013-11-01,7:30 pm,Milwaukee Bucks,105,Boston Celtics,98,Box Score,NaN,18624,NaN,False,False,False
21,2013-11-01,8:00 pm,Miami Heat,100,Brooklyn Nets,101,Box Score,NaN,17732,NaN,True,False,False
22,2013-11-01,7:00 pm,Cleveland Cavaliers,84,Charlotte Bobcats,90,Box Score,NaN,18017,NaN,True,False,True
23,2013-11-01,9:00 pm,Portland Trail Blazers,113,Denver Nuggets,98,Box Score,NaN,19155,NaN,False,False,False
24,2013-11-01,8:00 pm,Dallas Mavericks,105,Houston Rockets,113,Box Score,NaN,18142,NaN,True,True,True
25,2013-11-01,10:30 pm,San Antonio Spurs,91,Los Angeles Lakers,85,Box Score,NaN,18997,NaN,False,False,True


In [25]:
X_d = dataset[['VisitorLastWin','HomeLastWin']].values
clf = DecisionTreeClassifier(random_state =14)
scores = cross_val_score(clf , X_d , Y_true , scoring = 'accuracy')
np.mean(scores)*100

57.54365983295367

In [26]:
standing_file = './NBA_2013_Standings.csv'
standings = pd.read_csv(standing_file)
'''利用对阵队伍排名与上赛季最终排名比较得出HomeTeamRanksHeigher'''
dataset['HomeTeamRanksHeigher'] = 0
standings

,Rk,Team,Overall,Home,Road,E,W,A,C,SE,...,Post,≤3,≥10,Oct,Nov,Dec,Jan,Feb,Mar,Apr
0,1,San Antonio Spurs,62-20,32-9,30-11,24-6,38-14,8-2,7-3,9-1,...,24-5,7-0,35-12,1-0,13-3,11-4,8-6,9-3,16-0,4-4
1,2,Oklahoma City Thunder,59-23,34-7,25-16,23-7,36-16,8-2,8-2,7-3,...,16-11,10-4,33-8,1-0,10-3,14-3,13-4,6-5,10-4,5-4
2,3,Los Angeles Clippers,57-25,34-7,23-18,21-9,36-16,9-1,7-3,5-5,...,20-7,4-3,30-9,1-1,11-4,9-7,12-4,7-4,13-2,4-3
3,4,Indiana Pacers,56-26,35-6,21-20,38-14,18-12,15-3,12-4,11-7,...,16-14,4-3,29-17,2-0,13-1,10-4,10-5,9-3,8-10,4-3
4,5,Houston Rockets,54-28,33-8,21-20,23-7,31-21,7-3,8-2,8-2,...,18-11,7-6,30-9,1-0,12-5,8-8,10-4,8-2,10-4,5-5
5,6,Miami Heat,54-28,32-9,22-19,34-18,20-10,9-9,13-5,12-4,...,17-14,8-7,29-10,1-1,12-2,11-4,8-6,9-1,10-8,3-6
6,7,Portland Trail Blazers,54-28,31-10,23-18,23-7,31-21,9-1,9-1,5-5,...,18-11,8-9,24-13,0-1,13-2,12-4,8-6,7-5,8-9,6-1
7,8,Golden State Warriors,51-31,27-14,24-17,20-10,31-21,7-3,7-3,6-4,...,20-9,11-8,28-9,1-1,8-7,11-5,9-6,7-4,9-5,6-3
8,9,Memphis Grizzlies,50-32,27-14,23-18,21-9,29-23,6-4,7-3,8-2,...,21-9,10-4,18-13,0-1,8-7,5-9,12-3,7-5,12-5,6-2
9,10,Dallas Mavericks,49-33,26-15,23-18,20-10,29-23,5-5,9-1,6-4,...,17-11,11-11,23-10,1-0,9-8,8-5,9-8,9-3,8-6,5-3


In [27]:
for index , row in dataset.iterrows():
    home_team = row['Home Team']
    visitor_team = row['Visitor Team']
    home_rank = standings[standings['Team']==home_team]['Rk'].values[0]
    visitor_rank = standings[standings['Team']==visitor_team]['Rk'].values[0]
    dataset.at[index , "HomeTeamRanksHeigher"] = int(home_rank < visitor_rank)

In [28]:
X_d = dataset[['HomeTeamRanksHeigher','VisitorLastWin','HomeLastWin']].values
clf = DecisionTreeClassifier(random_state =14)
scores = cross_val_score(clf , X_d , Y_true , scoring = 'accuracy')
np.mean(scores)*100

67.0197418375095

In [29]:
last_match_winner = defaultdict(int)
row['HomeTeamWonLast'] = 0
teams_list = set()
"""找到两支队伍上次交锋情况 ， 不考虑主客队"""
for index , row in dataset.iterrows():
    home_team = row['Home Team']
    visitor_team = row['Visitor Team']
    teams = tuple(sorted([home_team , visitor_team]))
    teams_list.add(teams)
    dataset.at[index , "HomeTeamWonLast"] = 1 if last_match_winner[teams] == home_team else 0
    last_match_winner[teams] = home_team if row['Homewin'] else visitor_team
#dataset[(dataset['Visitor Team']=="Miami Heat")&(dataset['Home Team']=="San Antonio Spurs")|((dataset['Visitor Team']=="San Antonio Spurs")&(dataset['Home Team']=="Miami Heat"))]
X_d = dataset[['HomeTeamWonLast','HomeTeamRanksHeigher','VisitorLastWin','HomeLastWin']].values
clf = DecisionTreeClassifier(random_state =14)
scores = cross_val_score(clf , X_d , Y_true , scoring = 'accuracy')
np.mean(scores)*100

66.79195140470767